In [4]:
!pip install openai python-dotenv

In [5]:
import dotenv
import openai
import re
dotenv.load_dotenv('../.env')

True

In [6]:
hero_name = "Philippe"
story_idea = "une jeune sorcière en formation qui doit prouver ses compétences en sauvant son village d'un sortilège maléfique"
instructions = """
Consignes:
Un chapitre doit être court et se terminer par un cliffhanger.
Un Chapitre est divisé en plusieurs court paragraphes.
Un paragraphes contient Deux phrases au maximum."""
imagen_instructions = """-----
Write a very short one sentence visual description of the scene. Include a very short description of the character and its environnement. Keep it short and simple."""

In [7]:
def generate_prompt(story_idea, hero_name, instructions):
    prompt = f"""Ecris une histoire pour enfant composée de 5 chapitres.
{story_idea}
Personnage principal: {hero_name}
{instructions}
"""
    return prompt

In [8]:
def generate_story(story_idea, hero_name, instructions):
    prompt = generate_prompt(story_idea, hero_name, instructions)
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role":"user", "content": prompt}]
    )
    full_text = response.choices[0].message.content
    return full_text

In [9]:
story = generate_story(story_idea, hero_name, instructions)
print(story)

Chapitre 1: Léa et son rêve de devenir sorcière

Léa était une petite fille ordinaire, mais elle rêvait de devenir une sorcière puissante. Chaque nuit, elle s'endormait en imaginant des sorts spectaculaires et des potions magiques. Un matin, alors qu'elle se réveillait, une lettre mystérieuse glissa par la fente de sa porte. Elle était invitée à rejoindre l'école des sorciers pour y suivre sa formation. Léa était remplie de joie, mais elle savait qu'elle devait prouver ses compétences pour gagner sa place. Quelle épreuve l'attendait-elle ?

Chapitre 2: Le premier test inattendu

Léa se rendit à l'école des sorciers, pleine d'excitation et de nervosité. Elle fut accueillie par le professeur Merlin, qui lui dit qu'elle devait prouver sa valeur en sauvant son propre village d'un sortilège maléfique. Elle n'avait aucune idée de ce qui l'attendait, mais elle était déterminée à réussir. Elle se prépara pour sa première épreuve avec courage et détermination.

Chapitre 3: Le pouvoir des épreuv

In [10]:

def format_raw_story(story):

    # split story into chapters
    pattern = r"(Chapitre \d+: [^\n]+)\n\n(.*?)(?=\n\nChapitre|$)"

    # Find all matches
    story_by_chapters = re.findall(pattern, story, re.DOTALL)

    # Split the chapters into paragraphs of at least 100 characters
    story_by_chapters_and_paragraphs = []
    for chapter in story_by_chapters:
        chapter_title = chapter[0]
        chapter_content = chapter[1]
        chapter_content_splited = []
        for sentence in chapter_content.split("."):
            sentence = sentence.strip()
            if sentence == "": continue
            if len(chapter_content_splited) == 0:
                chapter_content_splited.append(sentence)
            elif len(chapter_content_splited[-1]) < 100:
                chapter_content_splited[-1] += f". {sentence}"
            else:
                chapter_content_splited.append(sentence)
        story_by_chapters_and_paragraphs.append((chapter_title, chapter_content_splited))
    
    return story_by_chapters_and_paragraphs

In [11]:
formated_story = format_raw_story(story)

In [12]:
formated_story

[('Chapitre 1: Léa et son rêve de devenir sorcière',
  ["Léa était une petite fille ordinaire, mais elle rêvait de devenir une sorcière puissante. Chaque nuit, elle s'endormait en imaginant des sorts spectaculaires et des potions magiques",
   "Un matin, alors qu'elle se réveillait, une lettre mystérieuse glissa par la fente de sa porte. Elle était invitée à rejoindre l'école des sorciers pour y suivre sa formation",
   "Léa était remplie de joie, mais elle savait qu'elle devait prouver ses compétences pour gagner sa place",
   "Quelle épreuve l'attendait-elle ?"]),
 ('Chapitre 2: Le premier test inattendu',
  ["Léa se rendit à l'école des sorciers, pleine d'excitation et de nervosité. Elle fut accueillie par le professeur Merlin, qui lui dit qu'elle devait prouver sa valeur en sauvant son propre village d'un sortilège maléfique",
   "Elle n'avait aucune idée de ce qui l'attendait, mais elle était déterminée à réussir. Elle se prépara pour sa première épreuve avec courage et déterminat

In [13]:
def text_to_images_desc(chapter_content):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role":"user", "content": f"""{chapter_content}
{imagen_instructions}"""}
                  ],
        max_tokens=50
    )
    full_text = response.choices[0].message.content
    return full_text

                   
    

In [14]:
story_with_images_desc = list()
for i, chapter in enumerate(formated_story):
    chapter_title = chapter[0]
    chapter_content = chapter[1]
    chapter_item = dict()
    chapter_item['chapter_title'] = chapter_title
    chapter_item['paragraphs'] = list()
    print(chapter_title)
    for j, paragraph_text in enumerate(chapter_content):
        paragraph_item = dict()
        paragraph_item['text'] = paragraph_text
        paragraph_item['image_desc'] = text_to_images_desc(paragraph_text)
        chapter_item['paragraphs'].append(paragraph_item)
    story_with_images_desc.append(chapter_item)
story_with_images_desc
        
        


Chapitre 1: Léa et son rêve de devenir sorcière


Chapitre 2: Le premier test inattendu
Chapitre 3: Le pouvoir des épreuves
Chapitre 4: La révélation des vérités
Chapitre 5: La victoire finale


[{'chapter_title': 'Chapitre 1: Léa et son rêve de devenir sorcière',
  'paragraphs': [{'text': "Léa était une petite fille ordinaire, mais elle rêvait de devenir une sorcière puissante. Chaque nuit, elle s'endormait en imaginant des sorts spectaculaires et des potions magiques",
    'image_desc': 'A little girl named Léa, surrounded by books of spells and cauldrons, dreams of becoming a powerful witch.'},
   {'text': "Un matin, alors qu'elle se réveillait, une lettre mystérieuse glissa par la fente de sa porte. Elle était invitée à rejoindre l'école des sorciers pour y suivre sa formation",
    'image_desc': "Une jeune femme se tient dans une chambre paisible à la lumière douce du matin, alors qu'une lettre mystérieuse glisse silencieusement sous sa porte."},
   {'text': "Léa était remplie de joie, mais elle savait qu'elle devait prouver ses compétences pour gagner sa place",
    'image_desc': 'Léa, a joyful individual, in a determined stance, ready to showcase her skills in order to 

# Gen images

# dl models

In [15]:
!mkdir -p models/image_encoder
!wget -P models https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter-plus-face_sdxl_vit-h.bin
!wget -O models/image_encoder/pytorch_model.bin https://huggingface.co/h94/IP-Adapter/resolve/main/models/image_encoder/pytorch_model.bin
!wget -O models/image_encoder/config.json       https://huggingface.co/h94/IP-Adapter/resolve/main/models/image_encoder/config.json

--2023-11-19 14:17:12--  https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter-plus-face_sdxl_vit-h.bin
Resolving huggingface.co (huggingface.co)... 52.84.90.33, 52.84.90.129, 52.84.90.106, ...
Connecting to huggingface.co (huggingface.co)|52.84.90.33|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/6f/e5/6fe5d8840a1ab462e1b9f52fbeabac6bb30febaf56fb5a439f08cca6d1cf4372/50e886d82940b3c5873d80c2b06d8a4b0d0fccec70bc44fd53f16ac3cfd7fc36?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27ip-adapter-plus-face_sdxl_vit-h.bin%3B+filename%3D%22ip-adapter-plus-face_sdxl_vit-h.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1700661985&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwMDY2MTk4NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy82Zi9lNS82ZmU1ZDg4NDBhMWFiNDYyZTFiOWY1MmZiZWFiYWM2YmIzMGZlYmFmNTZmYjVhNDM5ZjA4

In [16]:
!pip install accelerate diffusers==0.22.1 accelerate transformers einops git+https://github.com/tencent-ailab/IP-Adapter.git

  Cloning https://github.com/tencent-ailab/IP-Adapter.git to /tmp/pip-req-build-nyy26ql7
  Running command git clone --filter=blob:none --quiet https://github.com/tencent-ailab/IP-Adapter.git /tmp/pip-req-build-nyy26ql7
  Resolved https://github.com/tencent-ailab/IP-Adapter.git to commit e317f750e1829e846c944b9fdd7a2f2842c46501
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [5]:
import torch
from diffusers import StableDiffusionXLPipeline
from PIL import Image

from ip_adapter import IPAdapterPlusXL
from ip_adapter.custom_pipelines import StableDiffusionXLCustomPipeline


In [2]:
image = Image.open("my_phil_ip.png")

In [3]:
# load SDXL pipeline
pipe = StableDiffusionXLCustomPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    add_watermarker=False,
)
# load ip-adapter
ip_model = IPAdapterPlusXL(pipe, "models/image_encoder", 
                           "./models/ip-adapter-plus-face_sdxl_vit-h.bin", 
                           "cuda", num_tokens=16)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

pytorch_lora_weights.safetensors:   0%|          | 0.00/394M [00:00<?, ?B/s]

The config attributes {'skip_prk_steps': True} were passed to LCMScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.


AttributeError: 'UNet2DConditionModel' object has no attribute 'attn_processors'

In [5]:
def generate_picture(image, image_desc):
    prompt = f"{image_desc}, in a children cartoon style."
    images = ip_model.generate(pil_image=image, num_samples=1, num_inference_steps=30,
                           prompt=prompt, scale=0.4)
    return images[0]

In [23]:

def add_pictures_to_story(story_with_images_desc, image):

    for chapter in story_with_images_desc:
        print(f'done with chapter {chapter["chapter_title"]}')
        for paragraph in chapter['paragraphs']:
            paragraph['image'] = generate_picture(image, paragraph['image_desc'])
    return story_with_images_desc

In [24]:
story_with_images_desc = add_pictures_to_story(story_with_images_desc, image)

  0%|          | 0/30 [00:00<?, ?it/s]

done with chapter Chapitre 1: Léa et son rêve de devenir sorcière paragraph Léa était une petite fille ordinaire, mais elle rêvait de devenir une sorcière puissante. Chaque nuit, elle s'endormait en imaginant des sorts spectaculaires et des potions magiques


  0%|          | 0/30 [00:00<?, ?it/s]

done with chapter Chapitre 1: Léa et son rêve de devenir sorcière paragraph Un matin, alors qu'elle se réveillait, une lettre mystérieuse glissa par la fente de sa porte. Elle était invitée à rejoindre l'école des sorciers pour y suivre sa formation


  0%|          | 0/30 [00:00<?, ?it/s]

...

done with chapter Chapitre 5: La victoire finale paragraph Elle utilisa sa créativité et sa persévérance pour créer un sortilège de sauvetage. La cage de cristal commença à se fissurer puis à se briser en mille morceaux


  0%|          | 0/30 [00:00<?, ?it/s]

done with chapter Chapitre 5: La victoire finale paragraph Léa était libre et prête à affronter la sorcière jalouse une dernière fois. Elle lança son sortilège le plus puissant, emprisonnant la sorcière maléfique à son tour


  0%|          | 0/30 [00:00<?, ?it/s]

done with chapter Chapitre 5: La victoire finale paragraph Le village était sauvé et Léa avait prouvé qu'elle était une sorcière talentueuse. Elle était maintenant prête à continuer sa formation et à réaliser tous ses rêves magiques


# Create story album

In [1]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 3.4 MB/s eta 0:00:00 0:00:01


In [28]:
from docx import Document
from docx.shared import Inches
from io import BytesIO

In [32]:
document = Document()
for chapter in story_with_images_desc:
    document.add_heading(chapter['chapter_title'], level=1)
    for paragraph in chapter['paragraphs']:
        document.add_paragraph(paragraph["text"])
        image_buffer = BytesIO()
        paragraph['image'].save(image_buffer, format='png')
        document.add_picture(image_buffer, width=Inches(6.0))
        document.add_page_break()
document.save('story.docx')


# RUNPOD imagen

In [20]:
import dotenv
import os
import requests
import json
import base64
dotenv.load_dotenv('../functions/.env')

True

In [13]:
endpoint_url = "https://api.runpod.ai/v2/yfhto9s0koswbt/runsync"
endpoint_key = os.getenv("RUNPOD_API_KEY")
workflow = json.load(open("workflow.json", "r"))


In [14]:
# send post request to runpod api with bearer token and workflow
headers = {'Authorization': f'Bearer {endpoint_key}'}
response = requests.post(endpoint_url, headers=headers, json=workflow)

In [24]:
data_out = response.json()
b64_image = data_out['output']['message']
#save image
with open("comfy_image.png", "wb") as fh:
    fh.write(base64.b64decode(b64_image[len("data:image/png;base64,") :]))